In [171]:
import pandas as pd

car_data = pd.read_csv("datasets/car_prices.csv")
print(car_data.info)
print(car_data.columns)

car_data.drop(columns=["vin"], inplace=True) # not really meaningful

<bound method DataFrame.info of         year    make                model         trim       body  \
0       2015     Kia              Sorento           LX        SUV   
1       2015     Kia              Sorento           LX        SUV   
2       2014     BMW             3 Series   328i SULEV      Sedan   
3       2015   Volvo                  S60           T5      Sedan   
4       2014     BMW  6 Series Gran Coupe         650i      Sedan   
...      ...     ...                  ...          ...        ...   
558832  2015     Kia                 K900       Luxury      Sedan   
558833  2012     Ram                 2500  Power Wagon   Crew Cab   
558834  2012     BMW                   X5    xDrive35d        SUV   
558835  2015  Nissan               Altima        2.5 S      sedan   
558836  2014    Ford                F-150          XLT  SuperCrew   

       transmission                vin state  condition  odometer   color  \
0         automatic  5xyktca69fg566472    ca        5.0   1663

In [172]:
from sklearn.preprocessing import LabelEncoder


car_data["make"] = LabelEncoder().fit_transform(car_data["make"])
car_data["model"] = LabelEncoder().fit_transform(car_data["model"])
car_data["trim"] = LabelEncoder().fit_transform(car_data["trim"])
car_data["body"] = LabelEncoder().fit_transform(car_data["body"])
car_data["transmission"] = LabelEncoder().fit_transform(car_data["transmission"])
car_data["state"] = LabelEncoder().fit_transform(car_data["state"])
car_data["color"] = LabelEncoder().fit_transform(car_data["color"])
car_data["interior"] = LabelEncoder().fit_transform(car_data["interior"])
car_data["seller"] = LabelEncoder().fit_transform(car_data["seller"])


In [173]:
car_data["saledate"] = str(car_data["saledate"])
car_data["saledate"] = [date[21:25] for date in car_data["saledate"]]
print(car_data["saledate"])
car_data["saledate"] = LabelEncoder().fit_transform(car_data["saledate"])

0         2014
1         2014
2         2014
3         2014
4         2014
          ... 
558832    2014
558833    2014
558834    2014
558835    2014
558836    2014
Name: saledate, Length: 558837, dtype: object


In [174]:
print(car_data.corr())

                  year      make     model      trim      body  transmission  \
year          1.000000 -0.107049 -0.099938  0.007438 -0.007213     -0.003454   
make         -0.107049  1.000000  0.271055  0.124890  0.246417      0.034998   
model        -0.099938  0.271055  1.000000  0.234728  0.130787      0.031771   
trim          0.007438  0.124890  0.234728  1.000000  0.099927      0.000638   
body         -0.007213  0.246417  0.130787  0.099927  1.000000      0.030719   
transmission -0.003454  0.034998  0.031771  0.000638  0.030719      1.000000   
state         0.021518 -0.028043  0.016197  0.041061  0.012431      0.167334   
condition     0.340320 -0.013922 -0.041761 -0.013914 -0.003254      0.001375   
odometer     -0.772737  0.056959  0.098961  0.023365  0.027695     -0.014248   
color         0.064795  0.001842  0.002240  0.022380 -0.001657     -0.015870   
interior     -0.218824  0.016117  0.021063  0.016130 -0.006109     -0.009472   
seller        0.001966  0.126513  0.0310

It seems that the strongest predictors are year, condition, odometer, and mmr. 

In [175]:
df = car_data.drop(columns=["make", "model", "trim", "body", "transmission", "state", "color", "interior", "seller", "saledate"])
print(df.columns)

Index(['year', 'condition', 'odometer', 'mmr', 'sellingprice'], dtype='object')


In [178]:
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X = car_data.drop(columns=["sellingprice"])
y = car_data["sellingprice"]

imputer = SimpleImputer(missing_values=np.nan, strategy="most_frequent")
X = imputer.fit_transform(X)
std_scaler = StandardScaler()
X_scaled = std_scaler.fit_transform(X)

y = pd.DataFrame(y)
y = imputer.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=0.8, random_state=42)

In [183]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)
print(f'The model accounts for {round(lin_reg.score(X_train, y_train)*100, 2)}% of the variability')


The model accounts for 96.97% of the variability


In [187]:
def calculate_rmse(actual, pred):
    actual = np.array(actual)
    pred = np.array(pred)
    return np.sqrt(np.square(np.subtract(actual, pred)).mean())

y_pred = lin_reg.predict(X_test)
rmse = calculate_rmse(y_test, y_pred)
print(rmse)

1646.1526282677746


An RMSE value of 1,646 means that the predictions of this linear regression model can vary by approximately $1,650 from the actual price of the vehicle.

In [195]:
import tensorflow as tf

model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(32, input_shape=(14,), activation="relu"),
    tf.keras.layers.LeakyReLU(alpha=0.3),
    tf.keras.layers.Dense(64, activation="relu"),
    tf.keras.layers.LeakyReLU(alpha=0.5),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1), loss="mean_squared_error", metrics=["accuracy"])
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
13971/13971 [==============================] - 59s 4ms/step - loss: 3746646.7500 - accuracy: 8.9472e-06
Epoch 2/5
13971/13971 [==============================] - 54s 4ms/step - loss: 2973010.0000 - accuracy: 8.9472e-06
Epoch 3/5
13971/13971 [==============================] - 58s 4ms/step - loss: 2766561.0000 - accuracy: 8.9472e-06
Epoch 4/5
13971/13971 [==============================] - 53s 4ms/step - loss: 2702360.5000 - accuracy: 8.9472e-06
Epoch 5/5
13971/13971 [==============================] - 94s 7ms/step - loss: 2673949.0000 - accuracy: 8.9472e-06
